In [41]:
import os
import time

import requests
import pandas as pd
from bokeh.io import show, output_notebook, output_file, saving
from bokeh.embed import components
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import LinearAxis, Range1d, DatetimeTickFormatter, DatetimeTicker, ColumnDataSource, RangeTool, HoverTool, FixedTicker
from bokeh.palettes import Paired
from dotenv import load_dotenv
load_dotenv(".env");
output_notebook();

Loading BokehJS ...

In [2]:
token = os.environ["GITHUB_TOKEN"]
headers = {
    "Accept": "application/vnd.github.v3.star+json",
    "Authorization": f"token {token}"
}

In [3]:
url = "https://api.github.com/repos/psf/requests-html/stargazers"
resps = []

while True:
    resp = requests.get(url, headers=headers)
    resps.extend(resp.json())
    if "next" in resp.links:
        url = resp.links['next']['url']
        print(url)
    else:
        break

https://api.github.com/repositories/122775780/stargazers?page=2
https://api.github.com/repositories/122775780/stargazers?page=3
https://api.github.com/repositories/122775780/stargazers?page=4
https://api.github.com/repositories/122775780/stargazers?page=5
https://api.github.com/repositories/122775780/stargazers?page=6
https://api.github.com/repositories/122775780/stargazers?page=7
https://api.github.com/repositories/122775780/stargazers?page=8
https://api.github.com/repositories/122775780/stargazers?page=9
https://api.github.com/repositories/122775780/stargazers?page=10
https://api.github.com/repositories/122775780/stargazers?page=11
https://api.github.com/repositories/122775780/stargazers?page=12
https://api.github.com/repositories/122775780/stargazers?page=13
https://api.github.com/repositories/122775780/stargazers?page=14
https://api.github.com/repositories/122775780/stargazers?page=15
https://api.github.com/repositories/122775780/stargazers?page=16
https://api.github.com/repositori

https://api.github.com/repositories/122775780/stargazers?page=128
https://api.github.com/repositories/122775780/stargazers?page=129
https://api.github.com/repositories/122775780/stargazers?page=130
https://api.github.com/repositories/122775780/stargazers?page=131
https://api.github.com/repositories/122775780/stargazers?page=132
https://api.github.com/repositories/122775780/stargazers?page=133
https://api.github.com/repositories/122775780/stargazers?page=134
https://api.github.com/repositories/122775780/stargazers?page=135
https://api.github.com/repositories/122775780/stargazers?page=136
https://api.github.com/repositories/122775780/stargazers?page=137
https://api.github.com/repositories/122775780/stargazers?page=138
https://api.github.com/repositories/122775780/stargazers?page=139
https://api.github.com/repositories/122775780/stargazers?page=140
https://api.github.com/repositories/122775780/stargazers?page=141
https://api.github.com/repositories/122775780/stargazers?page=142
https://ap

https://api.github.com/repositories/122775780/stargazers?page=253
https://api.github.com/repositories/122775780/stargazers?page=254
https://api.github.com/repositories/122775780/stargazers?page=255
https://api.github.com/repositories/122775780/stargazers?page=256
https://api.github.com/repositories/122775780/stargazers?page=257
https://api.github.com/repositories/122775780/stargazers?page=258
https://api.github.com/repositories/122775780/stargazers?page=259
https://api.github.com/repositories/122775780/stargazers?page=260
https://api.github.com/repositories/122775780/stargazers?page=261
https://api.github.com/repositories/122775780/stargazers?page=262
https://api.github.com/repositories/122775780/stargazers?page=263
https://api.github.com/repositories/122775780/stargazers?page=264
https://api.github.com/repositories/122775780/stargazers?page=265
https://api.github.com/repositories/122775780/stargazers?page=266
https://api.github.com/repositories/122775780/stargazers?page=267
https://ap

In [4]:
df = pd.DataFrame(resps)

In [5]:
df.head()

,starred_at,user
0,2018-02-24T20:44:16Z,"{'login': 'jpic', 'id': 94636, 'node_id': 'MDQ..."
1,2018-02-24T20:52:30Z,"{'login': 'MechanisM', 'id': 121781, 'node_id'..."
2,2018-02-24T21:19:31Z,"{'login': 'guptarohit', 'id': 7895001, 'node_i..."
3,2018-02-24T21:52:14Z,"{'login': 'SenchoPens', 'id': 20418252, 'node_..."
4,2018-02-24T22:11:42Z,"{'login': 'jeanmarcalkazzi', 'id': 8341808, 'n..."


In [6]:
df.shape

(10185, 2)

In [7]:
df['dt'] = pd.to_datetime(df['starred_at'])

In [8]:
df_counts = df.groupby(df['dt'].dt.date).size().to_frame("count").reset_index()
df_counts['cumsum'] = df_counts['count'].cumsum()

In [52]:
t = df_counts['dt'].min()

In [61]:
import numpy as np

In [102]:
source = ColumnDataSource(data=df_counts)
colors = Paired[3]

p = figure(
    plot_height=200, plot_width=800, toolbar_location=None, tooltips=None,
#     x_range = [df_counts['dt'].min(), df_counts['dt'].max()],
    y_range=(df_counts['cumsum'].min(), df_counts['cumsum'].max()*1.05),
#     x_axis_type="datetime", 
    background_fill_color=colors[0], background_fill_alpha=.1,
    border_fill_color=colors[0], border_fill_alpha=.1,
    sizing_mode='scale_width',
    title="GitHubのスター数の遷移"
)

# 線グラフ
p.line(
    x='dt', y='cumsum', color=colors[1], line_width=5,
    legend="累計",
    source=source
)
p.xaxis.ticker = FixedTicker(ticks=[1, 2, 3, 4, 5])

# 増加数
p.extra_y_ranges = {"second": Range1d(start=df_counts['count'].min(), end=df_counts['count'].max())}
p.add_layout(LinearAxis(y_range_name="second"), 'right')
p.vbar(
    x='dt', top='count', width=1,
    y_range_name="second",
    color=colors[2],
    legend="1日ごと",
    source=source
)

# グラフの見た目を調整
p.axis.minor_tick_line_color = None
p.grid.visible = None
p.title.align = "center"
# p.xaxis.formatter = DatetimeTickFormatter(months=["%Y/%m"])

hover = HoverTool(
    tooltips=[("日付", "@dt{%F}"), ("累計", "@cumsum"), ("1日ごと", "@count")],
    formatters={"dt": 'datetime'},
    mode='vline'
)
p.add_tools(hover)

p.legend.location = "bottom_right"

show(p)
# saving.save(p, "test.html")
# script, plot_div = components(p)

In [103]:
df_counts['dt']

,dt,count,cumsum
0,2018-02-24,16,16
1,2018-02-25,1029,1045
2,2018-02-26,1831,2876
3,2018-02-27,948,3824
4,2018-02-28,1014,4838
5,2018-03-01,409,5247
6,2018-03-02,242,5489
7,2018-03-03,138,5627
8,2018-03-04,108,5735
9,2018-03-05,155,5890


# 書き出し

In [33]:
with open("GitHubTrend.html", mode="w") as f:
    f.write(plot_div + script)